# 🏥 Medical Image Segmentation - UNet Training (OPTIMIZED)

This notebook trains **UNet** models with speed optimizations:
- ✅ Mixed Precision (FP16) - 2x faster
- ✅ 20 epochs (vs 50) - sufficient for convergence
- ✅ Auto-save after each training
- ✅ Resume capability

**Runtime:** ~1.5 hours total (vs 3+ hours before)

---

## Step 1: Setup GPU Runtime

⚠️ **IMPORTANT:** Make sure you're using a GPU runtime!
- Go to: **Runtime → Change runtime type → GPU**

In [ ]:
# Verify GPU is available
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Training will be very slow.")

## Step 2: Clone Repository & Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/Prabhat9801/Medical-Image-Segmentation.git
%cd Medical-Image-Segmentation

# Install required packages
!pip install -q timm albumentations opencv-python-headless tqdm

## Step 3: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Step 4: Extract & Fix Data (Run Once)

In [ ]:
# Run the optimized data extraction script
!python colab_extract_data.py

## Step 5: Helper Function - Save to Drive After Each Training

In [ ]:
import os
import shutil
from datetime import datetime

def save_to_drive(model_name="unet"):
    """Save all experiments to Google Drive"""
    drive_dir = f"/content/drive/MyDrive/medical_segmentation_results/{model_name}_experiments"
    local_exp_dir = "/content/Medical-Image-Segmentation/experiments"
    
    # Create drive directory
    os.makedirs(drive_dir, exist_ok=True)
    
    # Copy all experiments
    if os.path.exists(local_exp_dir):
        for exp in os.listdir(local_exp_dir):
            src = os.path.join(local_exp_dir, exp)
            dst = os.path.join(drive_dir, exp)
            
            if os.path.isdir(src):
                # Remove old version if exists
                if os.path.exists(dst):
                    shutil.rmtree(dst)
                # Copy new version
                shutil.copytree(src, dst)
                print(f"✅ Saved: {exp}")
    
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"\n🎉 All results saved to Drive at {timestamp}")
    print(f"📁 Location: {drive_dir}")

def check_already_trained(model_name, data_fraction):
    """Check if this configuration was already trained"""
    drive_dir = f"/content/drive/MyDrive/medical_segmentation_results/{model_name}_experiments"
    
    if not os.path.exists(drive_dir):
        return False
    
    # Look for experiments with this model and data fraction
    frac_str = f"{int(data_fraction*100)}pct"
    for exp in os.listdir(drive_dir):
        if model_name in exp and frac_str in exp:
            print(f"⏭️  Skipping {model_name} {frac_str} - already trained!")
            print(f"   Found: {exp}")
            return True
    
    return False

print("✅ Helper functions loaded!")

## Step 6: Train UNet - 10% Data

**Expected time:** ~8 minutes

In [ ]:
if not check_already_trained("unet", 0.1):
    !python -m src.train \
        --model unet \
        --epochs 20 \
        --batch_size 24 \
        --data_fraction 0.1 \
        --lr 1e-4
    
    # Save immediately to Drive
    save_to_drive("unet")
    print("\n✅ UNet 10% training complete and saved!")
else:
    print("✅ UNet 10% already completed!")

## Step 7: Train UNet - 25% Data

**Expected time:** ~15 minutes

In [ ]:
if not check_already_trained("unet", 0.25):
    !python -m src.train \
        --model unet \
        --epochs 20 \
        --batch_size 24 \
        --data_fraction 0.25 \
        --lr 1e-4
    
    # Save immediately to Drive
    save_to_drive("unet")
    print("\n✅ UNet 25% training complete and saved!")
else:
    print("✅ UNet 25% already completed!")

## Step 8: Train UNet - 50% Data

**Expected time:** ~25 minutes

In [ ]:
if not check_already_trained("unet", 0.5):
    !python -m src.train \
        --model unet \
        --epochs 20 \
        --batch_size 24 \
        --data_fraction 0.5 \
        --lr 1e-4
    
    # Save immediately to Drive
    save_to_drive("unet")
    print("\n✅ UNet 50% training complete and saved!")
else:
    print("✅ UNet 50% already completed!")

## Step 9: Train UNet - 100% Data

**Expected time:** ~45 minutes

In [ ]:
if not check_already_trained("unet", 1.0):
    !python -m src.train \
        --model unet \
        --epochs 20 \
        --batch_size 24 \
        --data_fraction 1.0 \
        --lr 1e-4
    
    # Save immediately to Drive
    save_to_drive("unet")
    print("\n✅ UNet 100% training complete and saved!")
else:
    print("✅ UNet 100% already completed!")

## Step 10: Final Summary

In [ ]:
import os

drive_dir = "/content/drive/MyDrive/medical_segmentation_results/unet_experiments"

print("="*60)
print("🎉 UNet Training Complete!")
print("="*60)

if os.path.exists(drive_dir):
    experiments = [d for d in os.listdir(drive_dir) if os.path.isdir(os.path.join(drive_dir, d))]
    print(f"\n✅ Total experiments saved: {len(experiments)}")
    print(f"\n📁 Saved experiments:")
    for exp in sorted(experiments):
        print(f"   - {exp}")
    print(f"\n📍 Location: {drive_dir}")
else:
    print("\n⚠️ No experiments found in Drive")

print("\n" + "="*60)
print("Next steps:")
print("1. Run COLAB_TRAIN_UNETPP.ipynb to train UNet++")
print("2. Run COLAB_TRAIN_TRANSUNET.ipynb to train TransUNet")
print("3. Run COLAB_RESULTS.ipynb to evaluate all models")
print("="*60)